In [1]:
import os
import importlib

import pandas as pd
import numpy as np

In [29]:
# importing dymaic modules
from utils import preprocessing, io_files

importlib.reload(preprocessing)
importlib.reload(io_files)

<module 'utils.io_files' from '/media/tuhin/hdd/Documents/All_Python_Resources/AirBNB_DataAnalysis/utils/io_files.py'>

In [3]:
dataset_file = "dataset/archive/AB_NYC_2019.csv"

In [4]:
df0 = pd.read_csv(dataset_file)
df0.head()


print("Dataset size:\n"
      f"\trows: {df0.shape[0]}\n"
      f"\tcolumns: {df0.shape[1]}")

Dataset size:
	rows: 48895
	columns: 16


In [5]:
display(df0.columns.tolist())

['id',
 'name',
 'host_id',
 'host_name',
 'neighbourhood_group',
 'neighbourhood',
 'latitude',
 'longitude',
 'room_type',
 'price',
 'minimum_nights',
 'number_of_reviews',
 'last_review',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365']

In [6]:
preprocessing.drop_irrelevant_cols(df0)
missing_data_info_df = preprocessing.find_missing_data_freq(df0)
display(missing_data_info_df)

,Missing Data,% of Missing Data
last_review,10052,20.56
reviews_per_month,10052,20.56


### Finding Missing data for last_review or reviews_per_month

In [7]:
# rows with missing last-review and reviews per month
df_missing_data = df0[df0.last_review.isna() | df0.reviews_per_month.isna()]

display(df_missing_data.head())

print("Missing data rows:", df_missing_data.shape[0])

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
2,3647,4632,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
19,7750,17985,Manhattan,East Harlem,40.79685,-73.94872,Entire home/apt,190,7,0,NaN,NaN,2,249
26,8700,26394,Manhattan,Inwood,40.86754,-73.92639,Private room,80,4,0,NaN,NaN,1,0
36,11452,7355,Brooklyn,Bedford-Stuyvesant,40.68876,-73.94312,Private room,35,60,0,NaN,NaN,1,365
38,11943,45445,Brooklyn,Flatbush,40.63702,-73.96327,Private room,150,1,0,NaN,NaN,1,365


Missing data rows: 10052


We can see that there are **10,000+ rows** that have missing data.

The information of `last_review` or `reviews_per_month` is very important to determine the popularity or activity of certain listing.
\
So, we need to device a way to trim out those hosts that are least active and are going to do minimal contribution to the model.

In [16]:
unique_missing_data_hostid = df_missing_data.host_id.unique().tolist()
print(len(unique_missing_data_hostid), "unique hosts having missing missing information.")

8238 unique hosts having missing missing information.


Thus, `8238` hosts are _unique_ rest of the hosts from `10,052` listing hosts are _duplicate_.
\
This means, there're around 2k+ hosts that have repeated missing information about `last_review_date` or `reviews_per_month` data.

So, we are tring to find the ratio of $\frac{missing}{non-missing}$ data for each of those `8238` hosts.

In [38]:
# rows having host_id from unique_missing_data_hostid and their number of missing data rows/ number of non-missing data rows
rows_missing_last_review_reviews_per_month = df0[df0.host_id.isin(unique_missing_data_hostid)]
rows_missing_last_review_reviews_per_month.head()

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
2,3647,4632,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
19,7750,17985,Manhattan,East Harlem,40.79685,-73.94872,Entire home/apt,190,7,0,NaN,NaN,2,249
26,8700,26394,Manhattan,Inwood,40.86754,-73.92639,Private room,80,4,0,NaN,NaN,1,0
28,9518,31374,Manhattan,Inwood,40.86482,-73.92106,Private room,44,3,108,2019-06-15,1.11,3,311
36,11452,7355,Brooklyn,Bedford-Stuyvesant,40.68876,-73.94312,Private room,35,60,0,NaN,NaN,1,365


In [39]:
io_files.export_as_csv(rows_missing_last_review_reviews_per_month, filename="Missing-review-info-hostid-collection.csv")

PosixPath('/media/tuhin/hdd/Documents/All_Python_Resources/AirBNB_DataAnalysis/utils/tmp/Missing-review-info-hostid-collection.csv')

### Finding the information about missing/non-missing review data information.

Using this information, we can exclude certain hosts that have mostly missing data and can negatively affect the model prediction.
\
As per the calculation, above we will get 8238 rows of host id information that have **atleast one missing data row.**

The criteria for considering one missing data row is: **Either of the following condition has to satisfy**:
1. `number_of_reviews` must be `0` or `None/NaN`
2. `reviews_per_month` has to be `0` or `None/NaN`
3. `last_review` has to be `NaN`

In [41]:
df_host_missing_review = preprocessing.find_host_missing_review_data(df0)
df_host_missing_review.loc[df_host_missing_review.missing_review_count != 0, :]

,host_id,missing_review_count,non_missing_review_count,missing_review_percent
2,4632,1,0,100.000000
19,17985,1,1,50.000000
24,26394,1,0,100.000000
26,31374,1,2,33.333333
32,7355,1,0,100.000000
...,...,...,...,...
37452,274307600,1,0,100.000000
37453,274311461,1,0,100.000000
37454,274321313,1,0,100.000000
37455,23492952,1,0,100.000000


In [43]:
df_host_missing_review[df_host_missing_review.missing_review_percent >= 30].shape[0]

8068

**From above calculation, we can see that there are `8068` rows/hosts that contain `30% <=` amount of their data missing review information**